<a href="https://colab.research.google.com/github/Jaseem1911/Deep-Learning/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
from glob import glob
import pathlib 
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv3D,MaxPool3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
disease_path=pathlib.Path("/content/drive/MyDrive/Fish/data")

In [ ]:
zero_list=list(disease_path.glob("0/*.jpg"))          # *-shows all #(malignant/*)- shows all files inside the malignant folder
one_list=list(disease_path.glob("1/*.jpg"))
two_list=list(disease_path.glob("2/*.jpg"))
three_list=list(disease_path.glob("3/*.jpg"))
four_list=list(disease_path.glob("4/*.jpg"))
five_list=list(disease_path.glob("5/*.jpg"))

In [ ]:
len(zero_list),len(one_list),len(two_list),len(three_list),len(four_list),len(five_list)

(497, 587, 481, 495, 476, 412)

In [ ]:
disease_dict={"crab":zero_list,
              "shark":one_list,
              "penguin":two_list,
              "starfish":three_list,
              "seahorse":four_list,
              "seal":five_list}
disease_class={"crab":0,
              "shark":1,
               "penguin":2,
               "starfish":3,
               "seahorse":4,
               "seal":5
              }

In [ ]:
x=[]
y=[]

In [ ]:
for i in disease_dict:
  disease_name=i
  disease_path_list=disease_dict[disease_name]
  for path in disease_path_list:
    img=cv2.imread(str(path))
    img=cv2.resize(img,(224,224))
    img=img/255
    x.append(img)
    cls=disease_class[disease_name]
    y.append(cls)

In [ ]:
len(x)

2948

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.75,random_state=1)

In [ ]:
len(xtrain),len(ytrain),len(xtest),len(ytest)

(2211, 2211, 737, 737)

In [ ]:
xtrain.shape

(2211, 224, 224, 3)

In [ ]:
xtrain.shape,xtest.shape

((2211, 224, 224, 3), (737, 224, 224, 3))

In [ ]:
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
#base_model = tf.keras.applications.MobileNetV2(input_shape=(32, 32, 3),include_top=False,weights='imagenet')

In [ ]:
#print("[INFO] summary for base model...")
#print(base_model.summary())

In [ ]:
from tensorflow.keras.layers import MaxPooling2D
from keras.layers.core import Dropout
from tensorflow.keras import datasets, layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(6,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(xtrain, ytrain, epochs=35, 
                    validation_data=(xtest, ytest))

#model_hist=model.fit(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest),batch_size=180)

Epoch 1/35


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


70/70 [==============================] - 203s 3s/step - loss: 1.4745 - accuracy: 0.4265 - val_loss: 1.1143 - val_accuracy: 0.5631
Epoch 2/35
70/70 [==============================] - 201s 3s/step - loss: 1.0853 - accuracy: 0.5708 - val_loss: 1.2256 - val_accuracy: 0.4939
Epoch 3/35
70/70 [==============================] - 200s 3s/step - loss: 0.9492 - accuracy: 0.6359 - val_loss: 1.0738 - val_accuracy: 0.5970
Epoch 4/35
70/70 [==============================] - 200s 3s/step - loss: 0.8856 - accuracy: 0.6671 - val_loss: 1.1003 - val_accuracy: 0.5753
Epoch 5/35
70/70 [==============================] - 201s 3s/step - loss: 0.7136 - accuracy: 0.7291 - val_loss: 1.1187 - val_accuracy: 0.5943
Epoch 6/35
70/70 [==============================] - 200s 3s/step - loss: 0.7329 - accuracy: 0.7341 - val_loss: 1.0818 - val_accuracy: 0.6147
Epoch 7/35
70/70 [==============================] - 200s 3s/step - loss: 0.4446 - accuracy: 0.8367 - val_loss: 1.2220 - val_accuracy: 0.6323
Epoch 8/35
70/70 [======

In [ ]:
model.save("weight.h5")

In [ ]:
from tensorflow.keras.preprocessing import image
# testing the model
def testing_image(image_directory):
    test_image = image.load_img(image_directory, target_size = (224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    test_image = test_image/255
    result = model.predict(x= test_image)
    print(result)
    if np.argmax(result)  == 0:        
      prediction = 'Crab'
    elif np.argmax(result)  == 1:
      prediction = 'Shark'
    elif np.argmax(result)  == 2:
      prediction = 'Penguin'
    elif np.argmax(result)  == 3:
      prediction = 'Starfish'
    elif np.argmax(result)  == 4:
      prediction = 'Seahorse' 
  
    else:
      prediction = 'Seal'
    return prediction

In [ ]:
print(testing_image('/content/drive/MyDrive/Fish/Star.jpg'))

In [ ]:
from sklearn.metrics import  confusion_matrix

In [ ]:
Y_pred = model.predict_generator(xtest)

In [ ]:
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print('Confusion Matrix')

c=confusion_matrix(ytest, Y_pred)
#cm = confusion_matrix(np.where(ytest), Y_pred)